In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

# import os
# print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [2]:
from sklearn.model_selection import train_test_split

In [3]:
from __future__ import print_function, division
from builtins import range

In [4]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from gensim.models import KeyedVectors

In [5]:
from nltk.corpus import stopwords


In [6]:
ignore_words = set(stopwords.words('english')).union(set(stopwords.words('indonesian')))
print(len(ignore_words))

936


In [7]:
df_train=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')
df_cat=pd.read_json('categories.json')

In [8]:
df_train.title.head()



0                 nyx sex bomb pallete natural palette
1    etude house precious mineral any cushion pearl...
2                             milani rose powder blush
3                  etude house baby sweet sugar powder
4         bedak revlon color stay aqua mineral make up
Name: title, dtype: object

In [9]:
df_train['Group']=df_train.image_path.map(lambda x: x[:7])
df_test['Group']=df_test.image_path.map(lambda x: x[:7])

In [10]:
train,test=train_test_split(df_train,random_state=2019,stratify=df_train.Category,test_size=0.2)

In [11]:
np.random.seed(2019)

In [12]:
groups=['beauty_','mobile_','fashion']

In [13]:
# some linear models
from sklearn.linear_model import LogisticRegression, BayesianRidge

# SCM for classification
from sklearn.svm import SVC

from sklearn.pipeline import Pipeline

from sklearn.naive_bayes import MultinomialNB, GaussianNB, BernoulliNB

import warnings
warnings.filterwarnings('ignore')
np.random.seed(2019)


In [14]:
from sklearn.metrics import accuracy_score

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm

In [16]:
# let's build pipeline for parameter search in n_grams and and various models
lr=LogisticRegression()
nb=MultinomialNB()
svc=SVC()
# let's check first three and see which one is best
gnb=GaussianNB()
bnb=BernoulliNB()
br=BayesianRidge()
rf=RandomForestClassifier(n_estimators=150)

In [17]:
# Best cv scores for parameter for lr.


In [18]:
def submission(name,dic):
    df_sub=df_test.copy()
    for group in dic.keys():
        df_sub.loc[df_sub.Group==group,'Category']=dic[group]
    df_sub.loc[:,'Category']=df_sub.Category.astype('int')
    df_sub[['itemid','Category']].to_csv(name,index=False)

## Let's do ensembling now! 
## So we have results from:
## 1) CountVectorization with logistic regression : lr ending
## 2) CountVectorization with Naive Bayes:  nb ending
## 3) Image categorization:  im ending
## 4) word2vec predictions: w2v ending


## 1 and 2 might be highly correlated with each other. I might remove Naive Bayes. But we also can try with and without and observe results.


In [19]:
df_lr={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_test_proba_nlp_val_data2.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_nlp_all_data2.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_lr[groups[i]]=val_test

In [20]:
# own corpus in word2vec unlike original of this code
df_w2v={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_test_proba_w2v_own_val_data.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_w2v_own_all_data.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_w2v[groups[i]]=val_test

In [21]:
df_nb={}
for i in range(3):
    val_test=[]
    df1=pd.read_csv(groups[i]+'_val_proba_nlp_nb.csv')
    df2=pd.read_csv(groups[i]+'_test_proba_nlp_nb.csv')
    val_test.append(df1)
    val_test.append(df2)
    df_nb[groups[i]]=val_test

In [30]:
# We have previously ordered outputof image predictions so we can just upload all of them
df_im={}
for group in groups:
    if group=='mobile_':
        val_test=[]
        df1=pd.read_csv(group+'_val_im.csv') 
        df2=pd.read_csv(group+'_test_im.csv') 
        val_test.append(df1)
        val_test.append(df2)
        df_im[group]=val_test # Overite df_im but I use df_im in the rest of the code
    elif group=='fashion':
        val_test=[]
        df1=pd.read_csv('val_fashion_v2.csv') 
        df2=pd.read_csv('test_all_fashion_v2.csv') # trained on 80% of data
        val_test.append(df1)
        val_test.append(df2)
        df_im[group]=val_test # Overite df_im but I use df_im in the rest of the code
    else:
        val_test=[]
        df1=pd.read_csv('val_beauty__v2.csv') 
        df2=pd.read_csv('test_beauty__v2.csv') # trained on 80% of data
        val_test.append(df1)
        val_test.append(df2)
        df_im[group]=val_test # Overite df_im but I use df_im in the rest of the code


## We need to check impact of fashion image classification improvement on predictions.


# finally, we can start ensembling. Let's first check their relative error in internal test set. Also, correlations btw them

In [31]:
results=[df_lr,df_nb,df_w2v,df_im]

In [32]:
# Let's define columns for each category
columns={}
columns['beauty_']=[str(i) for i in range(17)]
columns['mobile_']=[str(i) for i in range(31,58)]
columns['fashion']=[str(i) for i in range(17,31)]

In [33]:
# df_im is actually df_im2 we overwrite it in the above code
models={'lr':df_lr,'nb':df_nb,'w2v':df_w2v,'im':df_im}

In [34]:
for name, model in models.items():
    if name=='im':
        pred='final_pred'
    else:
        pred='pred'
    print('Accuracy score for model: ', name)
    for group in groups:
        print(group,accuracy_score(test[test.Group==group].Category,model[group][0][pred]))

Accuracy score for model:  lr
beauty_ 0.789800582724
mobile_ 0.830687665679
fashion 0.652056166223
Accuracy score for model:  nb
beauty_ 0.764188635134
mobile_ 0.778949009824
fashion 0.592362486061
Accuracy score for model:  w2v
beauty_ 0.752324790202
mobile_ 0.776360517698
fashion 0.601351812658
Accuracy score for model:  im
beauty_ 0.69837918942
mobile_ 0.562763137377
fashion 0.525727680299


Previous results of word2vec based on combination of english and indonesian corpuses
Our own word2vec seems to be much better in all of groups

Accuracy score for model:  w2v
beauty_ 0.744054992411
mobile_ 0.766505535631
fashion 0.579777428825


In [35]:
df_all={}
for group in groups:
    val_test=[pd.DataFrame(),pd.DataFrame()]
    df_all[group]=val_test

In [36]:
len(models.keys())

4

In [37]:
#  Let's put all of the predictions together for ensembling
# I am still not sure about best stragery for ensembling
df_all_val={}
df_all_test={}
for group in groups:
    df_all_val[group]=models['lr'][group][0][columns[group]].copy()
    df_all_test[group]=models['lr'][group][1][columns[group]].copy()
    for name, model in models.items():
        if name!='lr':
            df1=models[name][group][0][columns[group]].copy()
            df_all_val[group]=df_all_val[group].join(df1,rsuffix=name)
            
            df2=models[name][group][1][columns[group]].copy()
            df_all_test[group]=df_all_test[group].join(df2,rsuffix=name)
        
# Now we put everything together.

In [38]:
# Sanity check
for group in groups:
    print("shape of df_all_val and test", group)
    print(df_all_val[group].shape, test[test.Group==group].shape)

shape of df_all_val and test beauty_
(57317, 68) (57317, 5)
shape of df_all_val and test mobile_
(32065, 108) (32065, 5)
shape of df_all_val and test fashion
(43941, 56) (43941, 5)


In [39]:
for group in groups:
    print("shape of df_all_test and df_test", group)
    print(df_all_test[group].shape, df_test[df_test.Group==group].shape)

shape of df_all_test and df_test beauty_
(76545, 68) (76545, 4)
shape of df_all_test and df_test mobile_
(40417, 108) (40417, 4)
shape of df_all_test and df_test fashion
(55440, 56) (55440, 4)


In [40]:
from sklearn.neighbors import KNeighborsClassifier

In [41]:
lr=LogisticRegression()
svm=SVC(kernel='linear')
rf=RandomForestClassifier(n_estimators=200)
knn=KNeighborsClassifier(n_neighbors=50)

In [42]:
pipe_en2={}
for group in groups:
    pipe_en2[group]=Pipeline([('model',rf)])

In [43]:
from sklearn.model_selection import StratifiedKFold

In [44]:
pipe_en2['beauty_']=RandomForestClassifier(n_estimators=280)

In [47]:
pipe_en2['beauty_']

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=280, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [48]:
# I am not sure but we might be overfitting. Let's try our previous approach


def eval_model():
    folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
    predictions2={}
    for group in ['beauty_']:
        print("Results for group", group)
        oof=np.zeros(len(df_all_val[group]))
        pred=np.zeros((len(df_test[df_test.Group==group]),folds.n_splits))
        X=df_all_val[group].values
        y=test[test.Group==group].Category.values
        X_sub=df_all_test[group].values
        model=pipe_en2[group]
        for fold_,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
            print("Fold:",fold_)
            X_train=X[trn_idx]
            y_train=y[trn_idx]
            model.fit(X_train,y_train)
            X_val=X[val_idx]
            oof[val_idx]=model.predict(X_val)
            pred[:,fold_]=model.predict(X_sub)
            print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
        print('Acuracy of overall:',accuracy_score(y,oof))
        predictions2[group]=pred
    return predictions2


In [49]:
pred=eval_model()

Results for group beauty_
Fold: 0
Acuracy of fold: 0.790496948561
Fold: 1
Acuracy of fold: 0.789271696468
Fold: 2
Acuracy of fold: 0.799703393527
Fold: 3
Acuracy of fold: 0.794189495725
Fold: 4
Acuracy of fold: 0.789560967094
Acuracy of overall: 0.792644416142


In [50]:
predictions2={}
for group in ['beauty_']:
    predictions2[group]=pd.DataFrame(pred[group]).mode(axis=1)[0].astype('int').values

In [51]:
for group in ['beauty_']:
    print(df_test[df_test.Group==group].shape,predictions2[group].shape)

(76545, 4) (76545,)


In [52]:
df_sub=pd.read_csv('ensemble_4_v1.csv')
df_sub.head()

,itemid,Category
0,370855998,5
1,637234604,5
2,690282890,5
3,930913462,5
4,1039280071,5


In [53]:
df_sub['Group']=df_test['Group']

In [54]:
# LB:0.76579. Here we use image classification model which is trained to 99.5% of dataset.
# Previous score: LB:0.76473  just slightly better than previous
for group in ['beauty_']:
    print(group)
    df_sub.loc[df_sub.Group==group,'Category']=predictions2[group]
del df_sub['Group']
df_sub.to_csv('ensemble_5.csv',index=False)

beauty_


In [58]:
# Now let's use model trained for all data set
df_im2

{}

In [57]:
df_all_test['fashion'].columns

Index(['17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28',
       '29', '30', '17nb', '18nb', '19nb', '20nb', '21nb', '22nb', '23nb',
       '24nb', '25nb', '26nb', '27nb', '28nb', '29nb', '30nb', '17w2v',
       '18w2v', '19w2v', '20w2v', '21w2v', '22w2v', '23w2v', '24w2v', '25w2v',
       '26w2v', '27w2v', '28w2v', '29w2v', '30w2v', '17im', '18im', '19im',
       '20im', '21im', '22im', '23im', '24im', '25im', '26im', '27im', '28im',
       '29im', '30im'],
      dtype='object')

In [43]:
# I am not sure but we might be overfitting. Let's try our previous approach


def eval_model():
    folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
    predictions2={}
    for group in ['fashion']:
        print("Results for group", group)
        oof=np.zeros(len(df_all_val[group]))
        pred=np.zeros((len(df_test[df_test.Group==group]),folds.n_splits))
        X=df_all_val[group].values
        y=test[test.Group==group].Category.values
        X_sub=df_all_test[group].values
        model=pipe_en2[group]
        for fold_,(trn_idx,val_idx) in enumerate(folds.split(X,y)):
            print("Fold:",fold_)
            X_train=X[trn_idx]
            y_train=y[trn_idx]
            model.fit(X_train,y_train)
            X_val=X[val_idx]
            oof[val_idx]=model.predict(X_val)
            pred[:,fold_]=model.predict(X_sub)
            print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
        print('Acuracy of overall:',accuracy_score(y,oof))
        predictions2[group]=pred
    return predictions2


In [44]:
pred=eval_model()

Results for group fashion
Fold: 0
Acuracy of fold: 0.6748180163785259
Fold: 1
Acuracy of fold: 0.6719372085087021
Fold: 2
Acuracy of fold: 0.6698521046643914
Fold: 3
Acuracy of fold: 0.6663252532149767
Fold: 4
Acuracy of fold: 0.6589226739551304
Acuracy of overall: 0.6683735008306593


In [48]:
predictions2={}
for group in ['fashion']:
    predictions2[group]=pd.DataFrame(pred[group]).mode(axis=1)[0].astype('int').values

In [49]:
for group in ['fashion']:
    print(df_test[df_test.Group==group].shape,predictions2[group].shape)

(55440, 4) (55440,)


In [50]:
df_sub=pd.read_csv('ensemble_3_v2.csv')
df_sub.head()

,itemid,Category
0,370855998,5
1,637234604,5
2,690282890,5
3,930913462,5
4,1039280071,5


In [53]:
df_sub['Group']=df_test['Group']

In [54]:
#LB:0.76473
# Previous score: LB:0.76428  just slightly better than previous
for group in ['fashion']:
    print(group)
    df_sub.loc[df_sub.Group==group,'Category']=predictions2[group]
del df_sub['Group']
df_sub.to_csv('ensemble_4.csv',index=False)

fashion


In [63]:
# We have previously ordered outputof image predictions so we can just upload all of them
df_im1={}
for group in groups:
    if group!='fashion':
        val_test=[]
        df1=pd.read_csv(group+'_val_im.csv') 
        df2=pd.read_csv(group+'_test_im.csv') 
        val_test.append(df1)
        val_test.append(df2)
        df_im1[group]=val_test
    else:
        val_test=[]
        df1=pd.read_csv('val_fashion_v2.csv') 
        df2=pd.read_csv('test_fashion_v2.csv') # trained on 80% of data
        val_test.append(df1)
        val_test.append(df2)
        df_im1[group]=val_test


In [64]:
df_im2={}
for group in groups:
    if group!='fashion':
        val_test=[]
        df1=pd.read_csv(group+'_val_im.csv') 
        df2=pd.read_csv(group+'_test_im.csv') 
        val_test.append(df1)
        val_test.append(df2)
        df_im2[group]=val_test
    else:
        val_test=[]
        df1=pd.read_csv('val_fashion_v2.csv') 
        df2=pd.read_csv('test_all_fashion_v2.csv') # Trained on 99.5% of data
        val_test.append(df1)
        val_test.append(df2)
        df_im2[group]=val_test

In [69]:
results=[df_lr,df_nb,df_w2v,df_im1]

In [70]:
# Let's define columns for each category
columns={}
columns['beauty_']=[str(i) for i in range(17)]
columns['mobile_']=[str(i) for i in range(31,58)]
columns['fashion']=[str(i) for i in range(17,31)]

In [71]:
# df_im is actually df_im2 we overwrite it in the above code
models={'lr':df_lr,'nb':df_nb,'w2v':df_w2v,'im':df_im1}

In [72]:
for name, model in models.items():
    if name=='im':
        pred='final_pred'
    else:
        pred='pred'
    print('Accuracy score for model: ', name)
    for group in groups:
        print(group,accuracy_score(test[test.Group==group].Category,model[group][0][pred]))

Accuracy score for model:  lr
beauty_ 0.7898005827241481
mobile_ 0.8306876656790894
fashion 0.6520561662228898
Accuracy score for model:  nb
beauty_ 0.7641886351344278
mobile_ 0.7789490098237954
fashion 0.5923624860608543
Accuracy score for model:  w2v
beauty_ 0.7523247902018598
mobile_ 0.7763605176984251
fashion 0.6013518126578822
Accuracy score for model:  im
beauty_ 0.6406476263586719
mobile_ 0.5627631373772025
fashion 0.5257276802985822


Previous results of word2vec based on combination of english and indonesian corpuses
Our own word2vec seems to be much better in all of groups

Accuracy score for model:  w2v
beauty_ 0.744054992411
mobile_ 0.766505535631
fashion 0.579777428825


In [73]:
df_all={}
for group in groups:
    val_test=[pd.DataFrame(),pd.DataFrame()]
    df_all[group]=val_test

In [74]:
len(models.keys())

4

In [75]:
#  Let's put all of the predictions together for ensembling
# I am still not sure about best stragery for ensembling
df_all_val={}
df_all_test={}
for group in groups:
    df_all_val[group]=models['lr'][group][0][columns[group]].copy()
    df_all_test[group]=models['lr'][group][1][columns[group]].copy()
    for name, model in models.items():
        if name!='lr':
            df1=models[name][group][0][columns[group]].copy()
            df_all_val[group]=df_all_val[group].join(df1,rsuffix=name)
            
            df2=models[name][group][1][columns[group]].copy()
            df_all_test[group]=df_all_test[group].join(df2,rsuffix=name)
        
# Now we put everything together.

In [76]:
pred=eval_model()

Results for group fashion
Fold: 0
Acuracy of fold: 0.6770928116469518
Fold: 1
Acuracy of fold: 0.6709134341940621
Fold: 2
Acuracy of fold: 0.6690557451649601
Fold: 3
Acuracy of fold: 0.6659838397632867
Fold: 4
Acuracy of fold: 0.660289260904225
Acuracy of overall: 0.6686693520857514


In [77]:
predictions2={}
for group in ['fashion']:
    predictions2[group]=pd.DataFrame(pred[group]).mode(axis=1)[0].astype('int').values

In [78]:
for group in ['fashion']:
    print(df_test[df_test.Group==group].shape,predictions2[group].shape)

(55440, 4) (55440,)


In [79]:
df_sub=pd.read_csv('ensemble_3_v2.csv')
df_sub.head()

,itemid,Category
0,370855998,5
1,637234604,5
2,690282890,5
3,930913462,5
4,1039280071,5


In [80]:
df_sub['Group']=df_test['Group']

In [81]:
#LB:0.0.76542 which is lower than previous results. Cuz images was trained to all data

for group in ['fashion']:
    print(group)
    df_sub.loc[df_sub.Group==group,'Category']=predictions2[group]
del df_sub['Group']
df_sub.to_csv('ensemble_4_v1.csv',index=False)

fashion


In [68]:
pipe_en=Pipeline([('model',lr)])

In [ ]:
# Previous function to evaluate model for beauty products
def eval_model_tr(model):
    y=df_beauty_val.Category
    folds=StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
    oof=np.zeros(len(df_all_prob))
    # prediction for the model with 80 of data
#     pred_80=np.zeros((len(X_test_beauty1),folds.n_splits))
    #prediction for the model with 100 of data
#     pred_100=np.zeros((len(X_test_beauty2),folds.n_splits))

    for fold_,(trn_idx,val_idx) in enumerate(folds.split(df_all_prob,y)):
#         print("Fold:",fold_)
        X_train=df_all_prob.loc[trn_idx,:]
        y_train=y[trn_idx]
        model.fit(X_train,y_train)
        X_val=df_all_prob.loc[val_idx,:]
        oof[val_idx]=model.predict(X_val)
#         pred_80[:,fold_]=model.predict(X_test_beauty1)
#         pred_100[:,fold_]=model.predict(X_test_beauty2)
#         print('Acuracy of fold:',accuracy_score(y[val_idx],oof[val_idx]))
    print('Acuracy of overall:',accuracy_score(y,oof))
    return accuracy_score(y,oof)

In [294]:
pd.merge(df,df_im['fashion'][0],on='index')

,itemid,index,31,32,33,34,35,36,37,38,...,50,51,52,53,54,55,56,57,final_pred,Category


In [219]:
#Let's check if it works
df_im['mobile_'][0].head()

,itemid,index,31,32,33,34,35,36,37,38,...,50,51,52,53,54,55,56,57,final_pred,Category
0,1058517290,3912586cdf51144db783b2bac1ee4d3f.jpg,0.026980,0.020451,0.004094,0.407791,0.372147,0.000012,0.001650,0.001870,...,0.005001,0.000660,0.001180,0.000943,0.000728,0.000967,0.000833,0.000563,34,34
1,1195506745,bb327fec3989b18f714deb3c07203fb8.jpg,0.072345,0.246230,0.057411,0.097349,0.273988,0.002055,0.027128,0.079219,...,0.000813,0.000252,0.000367,0.000301,0.000122,0.001252,0.000214,0.000445,35,35
2,1833821649,8959112348969556df29f24d06ecff8f.jpg,0.092200,0.167811,0.000182,0.012845,0.275278,0.000306,0.001374,0.000576,...,0.000136,0.002763,0.001903,0.000443,0.007212,0.000416,0.000406,0.000569,42,35
3,291297752,8733bba77a727e4771d51b5a8b48c8a5.jpg,0.043451,0.026404,0.017773,0.651762,0.116616,0.000137,0.004889,0.001703,...,0.000309,0.000054,0.000225,0.000511,0.000033,0.000112,0.000249,0.000087,34,34
4,1812460647,af7047829e497014daa5c2b4914a0fb4.jpg,0.872700,0.008431,0.000067,0.021418,0.066922,0.000281,0.000757,0.000065,...,0.000005,0.000052,0.000018,0.000056,0.000236,0.000021,0.000038,0.000015,31,31


In [220]:
df_train[df_train.Group=='mobile_'].head()

,itemid,title,Category,image_path,Group,index
506285,2346660,apple iphone 4s back glass spare part original...,31,mobile_image/a9c8f0fdd6587deed197634066cf7eee.jpg,mobile_,a9c8f0fdd6587deed197634066cf7eee.jpg
506286,2816338,iphone 4s 64gb white,31,mobile_image/3b9a11608551b11b9330268e0d055e01.jpg,mobile_,3b9a11608551b11b9330268e0d055e01.jpg
506287,2847602,samsung sm b310e piton dual sim,32,mobile_image/1d719e936841a83c165da620f927de68.jpg,mobile_,1d719e936841a83c165da620f927de68.jpg
506288,3116949,samsung caramel gt e1272 dual sim 32 mb putih,32,mobile_image/1d35a74d90df6cf4a02e6a5df9e9ff29.jpg,mobile_,1d35a74d90df6cf4a02e6a5df9e9ff29.jpg
506289,3794648,garskin sony experia z z1 z2 ultra,33,mobile_image/5556577b09539a9c0db0d00e0f171e2d.jpg,mobile_,5556577b09539a9c0db0d00e0f171e2d.jpg


In [139]:
?pd.merge

In [119]:
df.merge(df_im['mobile_'][0],on='index')

,itemid,index,31,32,33,34,35,36,37,38,...,50,51,52,53,54,55,56,57,final_pred,Category
0,2816338,3b9a11608551b11b9330268e0d055e01.jpg,0.883424,0.012605,0.000212,0.016350,0.073758,0.000144,0.000176,0.000084,...,2.546633e-05,1.471064e-05,0.000021,0.000019,0.000057,0.000040,0.000084,1.323019e-05,31,31
1,9503620,a23f0381039e5595559be27db3271d2f.jpg,0.278721,0.177533,0.003252,0.070696,0.063292,0.001435,0.000578,0.001302,...,3.675401e-04,1.966892e-04,0.000595,0.000358,0.000317,0.000214,0.001195,1.328025e-04,41,31
2,26702291,2c0ea46a95837e5171bd43512e82b1ce.jpg,0.001377,0.978527,0.000005,0.003402,0.013050,0.000007,0.000013,0.000019,...,2.433453e-07,1.700982e-06,0.000048,0.000004,0.000006,0.000010,0.000034,1.335207e-06,32,32
3,74050515,9a28a64c29e97e52c71ed45b029281ec.jpg,0.601693,0.033724,0.028193,0.028183,0.180513,0.002939,0.007077,0.011178,...,2.531332e-03,1.450678e-04,0.000201,0.000203,0.000186,0.000221,0.000318,3.438597e-04,31,31
4,122874123,3cfd98eb1f103623e12b8f081d702a66.jpg,0.048917,0.014367,0.001691,0.500844,0.248733,0.000180,0.003880,0.000100,...,7.084288e-05,2.756204e-05,0.000732,0.000258,0.000125,0.000090,0.000650,1.623501e-04,34,34
5,148667803,083faffd864dd1e6aff8cbc892ac15d7.jpg,0.623251,0.154122,0.004254,0.039050,0.051524,0.000153,0.003944,0.000892,...,1.371836e-05,4.439028e-04,0.000025,0.000118,0.000064,0.000020,0.000081,3.032195e-05,31,31
6,193655696,c1b8da227c7d1d4a4edd84496e914771.jpg,0.030702,0.029294,0.002633,0.014957,0.078660,0.802044,0.003124,0.009234,...,7.922183e-04,1.047876e-04,0.000475,0.002041,0.000055,0.000551,0.000276,1.658877e-04,36,36
7,210801578,f14091a8b8a09cd6b5e64b6ae2b45185.jpg,0.277034,0.214561,0.084110,0.074292,0.241911,0.001167,0.019214,0.004437,...,8.234581e-05,8.576331e-05,0.000280,0.003183,0.000413,0.000417,0.000455,1.556106e-04,31,32
8,212660404,39ed4573ea35e9b4e4619f37734c512a.jpg,0.014043,0.899937,0.000496,0.012193,0.026526,0.000140,0.000853,0.005278,...,2.121018e-05,4.890963e-04,0.000072,0.000279,0.000517,0.000105,0.000717,5.743321e-05,32,32
9,213418407,d961511fb22ca5eec825219035c0e1d4.jpg,0.005258,0.458902,0.001525,0.233880,0.271702,0.000182,0.001247,0.002195,...,3.200407e-05,4.720400e-04,0.000090,0.002059,0.000435,0.000022,0.000331,8.762698e-05,32,32


In [ ]:
for i in range(3):
    df=pd.DataFrame(predictions_proba[groups[i]])
    df.columns=classes[groups[i]]
    df['pred']=df.idxmax(axis=1)
    df['itemid']=df_test[df_test.Group==groups[i]].itemid.values
    df.to_csv(groups[i]+'_test_proba_nlp_all_data.csv',index=False)

In [72]:
y_pred=pipe.predict(X_train_EN)

In [74]:
accuracy_score(y_train,y_pred)

0.70672370657017336

In [ ]:
vectorizerEN = Word2VecVectorizerEN()

for group in groups:
    # English vectorization
    print('Results for group: ', group)
    vectorizerEN = Word2VecVectorizerEN()
    X_train_EN = vectorizerEN.fit_transform(train[train.Group==group].title)
    X_test_EN = vectorizerEN.transform(train[train.Group==group].title)
    
    # Indonesian vectorization
    vectorizerID = Word2VecVectorizerID()
    X_train_ID = vectorizerID.fit_transform(X_beauty_train)
    y_train = train[train.Group==group].Category.values
    X_test_ID = vectorizerID.transform(X_beauty_test)
    y_test = test[test.Group==group].Category.values
    
    # Combination of two models
    df_train_combined=pd.DataFrame(X_train_EN)
    df_train_combined=df_train_combined.join(pd.DataFrame(X_train_ID),rsuffix='ID')
    df_test_combined=pd.DataFrame(X_test_EN)
    df_test_combined=df_test_combined.join(pd.DataFrame(X_test_ID),rsuffix='ID')
    X_train_com=df_train_combined.values
    X_test_com=df_test_combined.values
    
    



    

In [81]:
#vectorizer = GloveVectorizer()
vectorizerEN = Word2VecVectorizerEN()
X_train_EN = vectorizerEN.fit_transform(X_beauty_train)
#Ytrain = y_beauty_train

X_test_EN = vectorizerEN.transform(X_beauty_test)
#Ytest = y_beauty_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 1639 / 200608
Numer of samples with no words found: 680 / 85975


In [82]:
X_train_EN.shape

(200608, 300)

In [83]:
#vectorizer = GloveVectorizer()
vectorizerID = Word2VecVectorizerID()
X_train_ID = vectorizerID.fit_transform(X_beauty_train)
y_train = y_beauty_train

X_test_ID = vectorizerID.transform(X_beauty_test)
y_test = y_beauty_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 164061 / 200608
Numer of samples with no words found: 70373 / 85975


In [84]:
X_train_ID.shape

(200608, 300)

In [96]:
df_train_combined=pd.DataFrame(X_train_EN)

In [99]:
df_train_combined=df_train_combined.join(pd.DataFrame(X_train_ID),rsuffix='ID')

In [100]:
df_test_combined=pd.DataFrame(X_test_EN)
df_test_combined=df_test_combined.join(pd.DataFrame(X_test_ID),rsuffix='ID')

In [103]:
df_train_combined.shape,df_test_combined.shape

((200608, 600), (85975, 600))

In [105]:
model = RandomForestClassifier(n_estimators=200)
model.fit(df_train_combined.values,y_train.values)
print("train score", model.score(df_train_combined.values,y_train.values))
print("test score:", model.score(df_test_combined.values,y_test.values))

train score 0.898877412665
test score: 0.730386740331


# Let's do the same for fashion 

In [106]:
#vectorizer = GloveVectorizer()
vectorizerID = Word2VecVectorizerID()
X_train_ID = vectorizerID.fit_transform(X_fashion_train)
y_train = y_fashion_train

X_test_ID = vectorizerID.transform(X_fashion_test)
y_test = y_fashion_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 122720 / 153791
Numer of samples with no words found: 52577 / 65911


In [107]:
#vectorizer = GloveVectorizer()
vectorizerEN = Word2VecVectorizerEN()
X_train_EN = vectorizerEN.fit_transform(X_fashion_train)
#Ytrain = y_beauty_train

X_test_EN = vectorizerEN.transform(X_fashion_test)
#Ytest = y_beauty_test

Loading in word vectors...
Finished loading in word vectors
Numer of samples with no words found: 144 / 153791
Numer of samples with no words found: 68 / 65911


In [108]:
X_train_EN.shape

(153791, 300)

In [109]:
X_train_ID.shape

(153791, 300)

In [110]:
df_train_combined=pd.DataFrame(X_train_EN)

In [111]:
df_train_combined=df_train_combined.join(pd.DataFrame(X_train_ID),rsuffix='ID')

In [112]:
df_test_combined=pd.DataFrame(X_test_EN)
df_test_combined=df_test_combined.join(pd.DataFrame(X_test_ID),rsuffix='ID')

In [113]:
df_train_combined.shape,df_test_combined.shape

((153791, 600), (65911, 600))

In [114]:
model = RandomForestClassifier(n_estimators=200)
model.fit(df_train_combined.values,y_train.values)
print("train score", model.score(df_train_combined.values,y_train.values))
print("test score:", model.score(df_test_combined.values,y_test.values))

train score 0.928266283463
test score: 0.560968578841


In [15]:
model = RandomForestClassifier(n_estimators=200)

In [16]:
model.fit(Xtrain,Ytrain)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [17]:
print("train score", model.score(Xtrain,Ytrain))
print("test score:", model.score(Xtest,Ytest))

train score 0.896150701866
test score: 0.731177667927


In [18]:
predictions = model.predict(Xtest)

In [19]:
from sklearn.metrics import classification_report,accuracy_score

In [20]:
print(classification_report(predictions,Ytest))

              precision    recall  f1-score   support

           0       0.30      0.60      0.40       585
           1       0.59      0.76      0.66      6726
           2       0.79      0.85      0.82      3282
           3       0.88      0.80      0.84     26695
           4       0.74      0.61      0.67     15794
           5       0.75      0.75      0.75     16427
           6       0.33      0.43      0.37       480
           7       0.70      0.74      0.72      3265
           8       0.51      0.59      0.54      1574
           9       0.37      0.72      0.49      1268
          10       0.44      0.59      0.50       246
          11       0.38      0.55      0.45       849
          12       0.85      0.73      0.79      7672
          13       0.43      0.63      0.51       588
          14       0.14      0.40      0.21       295
          15       0.12      0.43      0.19        47
          16       0.09      0.34      0.14       182

   micro avg       0.73   

In [21]:
print(accuracy_score(predictions,Ytest))

0.731177667927


In [ ]:
#from sklearn.linear_model import LogisticRegression

In [ ]:
#lr = LogisticRegression()

In [ ]:
#lr.fit(Xtrain,Ytrain)

In [ ]:
#predictions_lr = lr.predict(Xtest)

In [ ]:
#print(accuracy_score(predictions_lr,Ytest))